In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

from scipy.fftpack import fft, ifft
from scipy import signal
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

pd.set_option('precision', 30)
np.set_printoptions(precision = 30)


In [2]:
train_df = pd.read_csv('train/train.csv', dtype={'acoustic_data': np.int8, 'time_to_failure': np.float64})

MemoryError: 

In [ ]:
float_data = train_df.values


In [ ]:
def x_y_split(data):
    x = data[:,0]
    y = data[:,1]
    return x, y

def validation_split(data, ratio):
    slice_index = int(len(data) * ratio)
    train_data = data[:slice_index]
    valid_set = data[slice_index:]
    return train_data, valid_set

def create_segments(data):
    size = 150000
    segment = []
    for i in range(0, len(data), size):
        segment.append(data[i:i+size])
    return segment

# def generate_features(data):
#     feat = []
#     feat.append(data.mean())
#     feat.append(data.min())
#     feat.append(data.max())
# #     feat.append(data.median())
#     return np.array(feat)

def extract_features(z):
    return np.c_[z.mean(axis=0), 
                    z.min(axis=0),
                    z.max(axis=0),
                    z.std(axis=0)]

In [ ]:
# split training data into training and validationset
# train_set, valid_set = validation_split(float_data, 0.8)
X = float_data[:,0]
y = float_data[:,1]

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [6]:
class KerasDataGenerator(object):

    def __init__(self, data, n_steps=None, step_size=150000, batch_size=16, n_features=None):
        self.data = data
        self.n_steps = n_steps
        self.step_size = 150000
        self.current_pos = 0
        self.n_features = self.create_X().shape[1]
        self.batch_size = batch_size
    
    def reset_index(self):
        self.current_pos = 0

    def generator(self):
        
        while True:
            samples = np.zeros((self.batch_size, 1, self.n_features))
            targets = np.zeros(self.batch_size)
           
            for j in range(self.batch_size):
                samples[j] = self.create_X()
                try:
                    targets[j] = self.data[self.current_pos + self.step_size - 1, 1]
                except:
                    targets[j] = self.data[len(self.data) - 1, 1]
                self.current_pos = self.current_pos + self.step_size
                if self.current_pos >= len(self.data):
                    self.reset_index()
            yield samples, targets

    def create_X(self):
        x = self.data[self.current_pos:self.current_pos+self.step_size:,0]
#         x2 = x[-(self.step_size // 10):]
        return np.c_[extract_features(x),
                     extract_features(x[-(self.step_size // 10):]),
                     extract_features(x[-(self.step_size // 100):])]


In [7]:
# creates one batch of particular size
# a = train_gen.generator()
valid_gen = KerasDataGenerator(valid_set)
train_gen = KerasDataGenerator(train_set)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten

Using TensorFlow backend.


In [34]:
model = Sequential()

In [35]:
# model.add(Flatten())
model.add(LSTM(50,input_shape=(1,12), return_sequences=False))
model.add(Dense(1))

In [36]:
model.compile(loss='mae', optimizer='adam')

In [37]:
# model.output_shape

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 50)                12600     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________


In [38]:
batch_size = train_gen.batch_size
step_size = train_gen.step_size
steps_per_epoch = len(train_set)//(batch_size*step_size)
print(steps_per_epoch)
validation_steps = len(valid_set)//(batch_size*step_size)
validation_steps

209


52

In [ ]:
history = model.fit_generator(train_gen.generator(),
                              steps_per_epoch=steps_per_epoch,
                              epochs=100,
                              verbose=2,
                              validation_data=valid_gen.generator(),
                              validation_steps=validation_steps)


Epoch 1/100
 - 13s - loss: 3.9834 - val_loss: 3.6768
Epoch 2/100
 - 9s - loss: 3.2180 - val_loss: 3.4233
Epoch 3/100
 - 9s - loss: 3.0746 - val_loss: 3.3326
Epoch 4/100
 - 9s - loss: 3.0253 - val_loss: 3.2991
Epoch 5/100
 - 9s - loss: 3.0072 - val_loss: 3.2847
Epoch 6/100
 - 9s - loss: 2.9996 - val_loss: 3.2774
Epoch 7/100
 - 11s - loss: 2.9956 - val_loss: 3.2729
Epoch 8/100
 - 9s - loss: 2.9917 - val_loss: 3.2695
Epoch 9/100
 - 9s - loss: 2.9910 - val_loss: 3.2667
Epoch 10/100
 - 9s - loss: 2.9922 - val_loss: 3.2642
Epoch 11/100
 - 9s - loss: 2.9937 - val_loss: 3.2617
Epoch 12/100
 - 9s - loss: 2.9953 - val_loss: 3.2593
Epoch 13/100
 - 9s - loss: 2.9969 - val_loss: 3.2569
Epoch 14/100
 - 9s - loss: 2.9985 - val_loss: 3.2546
Epoch 15/100
 - 9s - loss: 3.0002 - val_loss: 3.2522
Epoch 16/100
 - 12s - loss: 3.0019 - val_loss: 3.2499
Epoch 17/100
 - 16s - loss: 3.0040 - val_loss: 3.2345
Epoch 18/100
 - 11s - loss: 3.0036 - val_loss: 3.2339
Epoch 19/100
 - 12s - loss: 3.0041 - val_loss: 3.2

In [ ]:

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
x = np.array([1,2,3,4,5,6])
x[3:10]